In [ ]:
from collections import OrderedDict, defaultdict
import math
import re
from IPython.display import Latex, display
import matplotlib.pyplot as plt
import sympy
from chempy import Substance, Equilibrium, Reaction, ReactionSystem
from chempy.kinetics.ode import get_odesys
from chempy.kinetics.rates import MassAction
from chempy.thermodynamics.expressions import EqExpr
from chempy.util.graph import rsys2graph
from chempy.util.pyutil import defaultkeydict
%matplotlib inline

In [ ]:
substances = OrderedDict([
    ('N', Substance('N', composition={'protein': 1}, latex_name='[N]')),
    ('U', Substance('U', composition={'protein': 1}, latex_name='[U]')),
    ('A', Substance('A', composition={'protein': 1}, latex_name='[A]')),
    ('L', Substance('L', composition={'ligand': 1}, latex_name='[L]')),
    ('NL', Substance('NL', composition={'protein': 1, 'ligand': 1}, latex_name='[NL]')),
])

In [ ]:
def _gibbs(args, T, R, backend, **kwargs):
    H, S, Cp, Tref = args
    H2 = H + Cp*(T - Tref)
    S2 = S + Cp*backend.log(T/Tref)
    return backend.exp(-(H2 - T*S2)/(R*T))

def _eyring(args, T, R, k_B, h, backend, **kwargs):
    H, S = args
    return k_B/h*T*backend.exp(-(H - T*S)/(R*T))

In [ ]:
Gibbs = EqExpr.from_callback(_gibbs, parameter_keys=('temperature', 'R'), argument_names=('H', 'S', 'Cp', 'Tref'))
Eyring = MassAction.from_callback(_eyring, parameter_keys=('temperature', 'R', 'k_B', 'h'), argument_names=('H', 'S'))

In [ ]:
thermo_dis = Gibbs(unique_keys=('He_dis', 'Se_dis', 'Cp_dis', 'Tref_dis'))
thermo_u = Gibbs(unique_keys=('He_u', 'Se_u', 'Cp_u', 'Tref_u'))  # ([He_u_R, Se_u_R, Cp_u_R, Tref])
kinetics_agg = Eyring(unique_keys=('Ha_agg', 'Sa_agg'))  # EyringMassAction([Ha_agg, Sa_agg])
kinetics_as = Eyring(unique_keys=('Ha_as', 'Sa_as'))
kinetics_f = Eyring(unique_keys=('Ha_f', 'Sa_f'))

In [ ]:
eq_dis = Equilibrium({'NL'}, {'N', 'L'}, thermo_dis, name='ligand-protein dissociation')
eq_u = Equilibrium({'N'}, {'U'}, thermo_u, {'L'}, {'L'}, name='protein unfolding')
r_agg = Reaction({'U'}, {'A'}, kinetics_agg, {'L'}, {'L'}, name='protein aggregation')

In [ ]:
rsys = ReactionSystem(
    eq_dis.as_reactions(kb=kinetics_as, new_name='ligand-protein association') +
    eq_u.as_reactions(kb=kinetics_f, new_name='protein folding') +
    (r_agg,), substances, name='4-state CETSA system')
# rsys

In [ ]:
vecs, comp = rsys.composition_balance_vectors()
names = rsys.substance_names()
dict(zip(comp, [dict(zip(names, v)) for v in vecs]))

In [ ]:
rsys2graph(rsys, '4state.png', save='.', include_inactive=False)
from IPython.display import Image; Image('4state.png')

In [ ]:
from IPython.display import HTML
HTML('1st order processes\n' + rsys.unimolecular_html_table()[0] +
     '<br><br>2nd order processes\n' + rsys.bimolecular_html_table()[0])

In [ ]:
def pretty_replace(s, subs=None):
    if subs is None:
        subs = {
            'Ha_(\w+)': r'\Delta_{\1}H^{\\neq}',
            'Sa_(\w+)': r'\Delta_{\1}S^{\\neq}',
            'He_(\w+)': r'\Delta_{\1}H^\circ',
            'Se_(\w+)': r'\Delta_{\1}S^\circ',
            'Cp_(\w+)': r'\Delta_{\1}\,C_p',
            'Tref_(\w+)': r'T^{\circ}_{\1}',
        }
    for pattern, repl in subs.items():
        s = re.sub(pattern, repl, s)
    return s

def mk_Symbol(key):
    if key in substances:
        arg = substances[key].latex_name
    else:
        arg = pretty_replace(key.replace('temperature', 'T'))

    return sympy.Symbol(arg)

autosymbols = defaultkeydict(mk_Symbol)
rnames = {}
for rxn in rsys.rxns:
    rnames[rxn.name] = rxn.name.replace(' ', '~').replace('-','-')
    rate_expr_str = sympy.latex(rxn.rate_expr()(autosymbols, backend=sympy, reaction=rxn))
    lstr = r'$r(\mathrm{%s}) = %s$' % (rnames[rxn.name], rate_expr_str)
    display(Latex(lstr))

In [ ]:
ratexs = [autosymbols['r(\mathrm{%s})' % rnames[rxn.name]] for rxn in rsys.rxns]
rates = rsys.rates(autosymbols, backend=sympy, ratexs=ratexs)
for k, v in rates.items():
    display(Latex(r'$\frac{[%s]}{dt} = %s$' % (k, sympy.latex(v))))

In [ ]:
odesys, pk, unique, p_units = get_odesys(rsys, include_params=False)

In [ ]:
params = dict(
    R=8.314472,  # or N_A & k_B
    k_B=1.3806504e-23,
    h=6.62606896e-34,  # k_B/h == 2.083664399411865e10 K**-1 * s**-1
    He_dis=-45e3,
    Se_dis=-400,
    Cp_dis=1.78e3,
    Tref_dis=298.15,
    He_u=60e3,
    Cp_u=20.5e3,
    Tref_u=298.15,
    Ha_agg=106e3,
    Sa_agg=70,
    Ha_as=4e3,
    Sa_as=-10,
    Ha_f=90e3,
    Sa_f=50,
    temperature=50 + 273.15
)

def Se0_from_Tm(Tm):
    dH0, T, T0, dCp = params['He_u'], params['temperature'], params['Tref_u'], params['Cp_u']
    return dH0/T + (T-T0)*dCp/T - math.log(T/T0)
params['Se_u'] = Se0_from_Tm(48.2+273.15)

In [ ]:
c0 = defaultdict(float, {'N': 1e-9, 'L': 1e-8})
tout, Cout, info = odesys.integrate(3600*24, c0, params, integrator='cvode', nsteps=9000,
                                    atol=1e-11, rtol=1e-11, first_step=1e-90)
_ = odesys.plot_result(xscale='linear', yscale='log')
_ = plt.legend(loc='best')